# Spotify Wrapped

### Install the following libraries

- pip install spotipy
- pip install gspread

###### Add '!' before pip if you are using Jupyter Notebook

### Spotify API 
- (https://developer.spotify.com/documentation/web-api)

### Spotipy Library Documentation
- (https://spotipy.readthedocs.io/en/2.18.0/)

In [ ]:
CLIENT_SERVER_ID = "your-api" # add your client server id from spotify api
CLIENT_SECRET_ID = "your-secret-id" # add your client secret id from spotify api
REDIRECT_ID = "http://127.0.0.1:8080" # you can add any localhost address
SCOPE = "user-top-read"

In [ ]:
import time
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyOAuth

In [ ]:
# this will ask you to connect your spotify account

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_SERVER_ID, client_secret=CLIENT_SECRET_ID,redirect_uri=REDIRECT_ID, scope=SCOPE))

In [ ]:
# 3 time ranges, short_term, medium_term, long_term. Limit gives you number of songs

top_tracks_short = sp.current_user_top_tracks(limit=10, offset=0, time_range="short_term")

In [ ]:
top_tracks

In [ ]:
for track in top_tracks['items']:
    print(track['album']) 

In [ ]:
# this function return ids of songs

def track_song_id(tracks):
    track_ids = []
    
    for i in tracks['items']:
        track_ids.append(i['id'])
        
    return track_ids

In [ ]:
track_id_short = track_song_id(top_tracks)

In [ ]:
track_id_short

In [ ]:
# this function extract features using the id

def track_features(id):
    data = sp.track(id)
    
    name = data['name']
    artist = data['album']['artists'][0]['name']
    album = data['album']['name']
    spotify_url = data['external_urls']['spotify']
    album_cover = data['album']['images'][0]['url']
    
    info = [name, artist, album, spotify_url, album_cover]
    
    return info

In [ ]:
tracks = []

for i in range(len(track_id_short)):
    time.sleep(.5)
    track = track_features(track_id_short[i])
    tracks.append(track)

In [ ]:
tracks

# Convert the data into a DataFrame

In [ ]:
df = pd.DataFrame(tracks, columns = ['name', 'artist', 'album', 'spotify_url', 'album_cover'])

df

# Google Spreadsheet

#### If you want to export your data to Google Spreadsheet, you can do so by running the following code.

- But first you need to go to gspread doc and follow the instructions (https://docs.gspread.org/en/latest/oauth2.html#enable-api-access)

In [ ]:
import gspread

In [ ]:
gc = gspread.service_account(filename="spotify_wrapped_gspread.json")

In [ ]:
sh = gc.open("My Spotify Wrapped Data")

In [ ]:
def df_to_gspread(track_ids):
    tracks = []

    for i in range(len(track_ids)):
        time.sleep(0.5)
        track = track_features(track_ids[i])
        tracks.append(track)

    # Create DataFrame
    df = pd.DataFrame(tracks, columns=['name', 'artist', 'album', 'spotify_url', 'album_cover'])

    # Authenticate with Google Sheets
    gc = gspread.service_account(filename="spotify_wrapped_gspread.json")
    sh = gc.open("My Spotify Wrapped Data")

    # Access worksheet
    worksheet = sh.worksheet(f"{time_period}")

    # Update worksheet with headers and data
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())

    print("Done!")

In [ ]:
time_ranges = ["short_term", "medium_term", "long_term"]

for time_period in time_ranges:
    top_tracks = sp.current_user_top_tracks(limit=10, offset=0, time_range=time_period)
    track_ids = track_song_id(top_tracks)
    df_to_gspread(track_ids)